In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
print(hasattr(cv2, 'ximgproc'))  # True ならOK

True


In [2]:
img1 = cv2.imread('data/IMG_8297.jpg')
img2 = cv2.imread('data/IMG_8298.jpg')
img1 = cv2.resize(img1, ((int)(img1.shape[1]/4), (int)(img1.shape[0]/4)))
h,w, c = img1.shape

# resize img2 to match img1 
img2= cv2.resize(img2, (w, h), interpolation=cv2.INTER_LINEAR)

img12 = np.hstack((img1, img2))

cv2.namedWindow('OriginalImage', cv2.WINDOW_NORMAL)
cv2.imshow('OriginalImage', img12)
cv2.waitKey(0)

-1

In [3]:
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

img1_kp = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
img2_kp = cv2.drawKeypoints(img2, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

img12_kp = np.hstack((img1_kp, img2_kp))
cv2.namedWindow('FeaturePoints', cv2.WINDOW_NORMAL)
cv2.imshow('FeaturePoints', img12_kp)
cv2.waitKey(0)

-1

In [4]:
bf = cv2.BFMatcher()
matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x:x.distance)

img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:200], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('MatchedPoints', cv2.WINDOW_NORMAL)
cv2.imshow('MatchedPoints', img_matches)
cv2.waitKey(0)

-1

### 最近隣距離比により、良いマッチングペアを選出する

In [5]:
matches_rt = bf.knnMatch(des1, des2, k=2)
good_matches = []
for m,n in matches_rt:
    if m.distance < 0.75*n.distance:
        good_matches.append(m)

good_matches = sorted(good_matches, key = lambda x:x.distance)
img_good_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches[:200], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('MatchedPoints200', cv2.WINDOW_NORMAL)
cv2.imshow('MatchedPoints200', img_good_matches)
cv2.waitKey(0)

-1

### 最近距離比から選出した良いマッチングペアにより基礎行列Fを計算する。算出したFにより外れポイントを外す

In [6]:
pts1 = []
pts2 = []

for i,m in enumerate(good_matches):
    pts1.append(kp1[m.queryIdx].pt)
    pts2.append(kp2[m.trainIdx].pt)

import numpy as np
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.RANSAC)

pts1_inliers = pts1[mask.ravel() == 1]
pts2_inliers = pts2[mask.ravel() == 1]

good_matches_inliers = [m for i,m in enumerate(good_matches[:200]) if mask[i,0] == 1]

img_good_matches_inliers = cv2.drawMatches(img1,kp1,img2,kp2,good_matches_inliers[:200],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.namedWindow('GoodMatchedPoints', cv2.WINDOW_NORMAL)
cv2.imshow('GoodMatchedPoints', img_good_matches_inliers)
cv2.waitKey(0)


-1

### エピポーラ線を描画する。
- エピポーラ線 ax+by+c=0の(a,b,c)は(r[0],r[1],[r[2]])
- エピポーラ線の両端はx=0の時とx=画像幅の時

In [7]:
def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    r,c,ch = img1.shape
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = cv2.line(img1, (x0,y0), (x1,y1), color,1)
        img1 = cv2.circle(img1,tuple(pt1),5,color,-1)
        img2 = cv2.circle(img2,tuple(pt2),5,color,-1)
    return img1,img2

pts1_inliers_200 = pts1_inliers[0:200,:]
pts2_inliers_200 = pts2_inliers[0:200,:]

lines1 = cv2.computeCorrespondEpilines(pts2_inliers_200.reshape(-1,1,2),2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1.copy(),img2.copy(),lines1,pts1,pts2)
lines2 = cv2.computeCorrespondEpilines(pts1_inliers_200.reshape(-1,1,2),1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2.copy(),img1.copy(),lines2,pts2,pts1)
img35 = np.hstack((img3,img5))
cv2.namedWindow('EpipoleLines', cv2.WINDOW_NORMAL)
cv2.imshow('EpipoleLines', img35)
cv2.waitKey(0)

-1

### さらに、Fで外れ値を削除した後の200個のペアを選んでにより基礎行列Fを計算して、エピポーラ線を可視化する

In [8]:
F, mask = cv2.findFundamentalMat(pts1[0:200,:], pts2[0:200,:], cv2.FM_RANSAC)

pts1 = pts1[0:200,:]
pts2 = pts2[0:200,:]
pts1_inliers = pts1[mask.ravel() == 1]
pts2_inliers = pts2[mask.ravel() == 1]

pts1_inliers_200 = pts1_inliers[0:200,:]
pts2_inliers_200 = pts2_inliers[0:200,:]

lines1 = cv2.computeCorrespondEpilines(pts2_inliers_200.reshape(-1,1,2),2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawlines(img1.copy(),img2.copy(),lines1,pts1,pts2)
lines2 = cv2.computeCorrespondEpilines(pts1_inliers_200.reshape(-1,1,2),1,F)
lines2 = lines2.reshape(-1,3)
img3,img4 = drawlines(img2.copy(),img1.copy(),lines2,pts2,pts1)
img35 = np.hstack((img3,img5))
cv2.namedWindow('GoodEpipoleLines', cv2.WINDOW_NORMAL)
cv2.imshow('GoodEpipoleLines', img35)
cv2.waitKey(0)

-1

## F から整列するホモグラフィ変換Ｈ１，Ｈ２を得て、整列後の画像枠を計算する

In [9]:
h, w ,c= img1.shape
retval, H1, H2 = cv2.stereoRectifyUncalibrated(
    np.float32(pts1[mask.ravel() == 1]),
    np.float32(pts2[mask.ravel() == 1]),
    F, imgSize=(w, h)
)

imgL_rectify = cv2.warpPerspective(img1, H1, (w, h))
imgR_rectify = cv2.warpPerspective(img2, H2, (w, h))

## ステレオマッチングでStereoデプス推定
https://cvml-expertguide.net/terms/cv/camera-geometry/stereo-matching/


In [44]:
# 5. SGBMステレオマッチング
min_disp = 0
num_disp = 16 * 5  # 16の倍数
block_size = 6

stereo = cv2.StereoSGBM_create(
    minDisparity=min_disp,
    numDisparities=num_disp,
    blockSize=block_size,
    P1=8 * 1 * block_size ** 2,
    P2=32 * 1 * block_size ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32
)
disparity = stereo.compute(imgL_rectify, imgR_rectify).astype(np.float32) / 16.0
disparity = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

cv2.namedWindow('DisparityMap(SGBM)', cv2.WINDOW_NORMAL)
cv2.imshow("DisparityMap(SGBM)", disparity)
cv2.waitKey(0)


-1

In [48]:
# uv add opencv-contrib-python for cv2.filterSpeckles
disparity_filtered = np.uint8(disparity)
cv2.filterSpeckles(disparity_filtered, newVal=0, maxSpeckleSize=100, maxDiff=16*5)

# 正規化して表示用に変換
disparity_filtered = cv2.normalize(disparity_filtered, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

cv2.namedWindow('DisparityMap(Filtered)', cv2.WINDOW_NORMAL)
cv2.imshow("DisparityMap(Filtered)", np.hstack((disparity,disparity_filtered)))
cv2.waitKey(0)

-1

## エネルギー関数最小化デプスマップ改善
use [ uv add PyMaxflow ] to add library
下記のコードはエラーになる。原因は、pythonのバージョンとmaxflowのバージョンが合わないと推測。

In [39]:

import maxflow
import numpy as np


# 欠損値やノイズを除去
disparity[disparity < 0] = 0

# データ項：視差マップの中央値との差を使う
disp_median = np.median(disparity)
data_cost = np.abs(disparity - disp_median)

# サイズ
h, w = disparity.shape

# グラフ構築
g = maxflow.Graph[float]()
nodes = g.add_grid_nodes((h, w))

# Smoothness: 画素間のスムーズネス（定数罰則）
structure = np.array([[0, 1, 0],
                      [1, 0, 1],
                      [0, 1, 0]])

smoothness_weight = 5.0
g.add_grid_edges(nodes, smoothness_weight, structure=structure, symmetric=True)

# Data term: 各画素にターミナルエッジ
# ここではラベル0：median近い（前景）、ラベル1：外れ値（背景）とみなす
data_term_foreground = data_cost          # 視差が中央値に近いとコスト小 → foreground
data_term_background = 1.0 - data_cost    # 離れてると背景（単純化）

# 正規化（0〜1）
data_term_foreground = (data_term_foreground - data_term_foreground.min()) / (data_term_foreground.max() - data_term_foreground.min())
data_term_background = 1.0 - data_term_foreground

# ターミナルエッジ追加
g.add_grid_tedges(nodes, data_term_foreground, data_term_background)

# 最小カット実行
flow = g.maxflow()
segments = g.get_grid_segments(nodes)

# 最終ラベルマップ作成
optimized_mask = np.int32(~segments)  # True: foreground, False: background

# 改善視差マップ：foreground のみ保持、背景はゼロに
refined_disparity = disparity * optimized_mask

cv2.namedWindow('graph_cuts', cv2.WINDOW_NORMAL)
cv2.imshow("graph_cuts", refined_disparity)
cv2.waitKey(0)

AttributeError: module 'maxflow' has no attribute 'Graph'